In [3]:
import pandas as pd
import numpy as np
import jqdatasdk as jq
import os

In [4]:
jq.auth('13918852005','960312Lsc')
# jq.auth('15821912507','912507')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [3]:
# 沪深两市自2011年1月1日前上市股票列表
hslslist = list(jq.get_all_securities(types=['stock'], date='2011-01-01').index)
stock_download_len = len(hslslist)

In [4]:
print(stock_download_len)

2042


In [10]:
# stock_download_lst = pd.read_csv('E:/Stock_Data/stock_code.csv')
# stock_download_lst = list(stock_download_lst.iloc[:,0])
# stock_download_len = len(stock_download_lst)

In [4]:
trade_days = jq.get_trade_days(start_date='2011-01-01', end_date='2018-12-31')
trade_days_len = len(trade_days)

In [5]:
used_stock_list = []
cnt_code = 0
code_list_temp = os.listdir('E:/Stock_Data/stock_return_data')
code_list = [i[:-4] for i in code_list_temp]

for code in hslslist: # 从stock_download_lst变为了hslslist，收益率数据要求更多，不一定要完整年报
    
    if code in code_list:
        cnt_code += 1
        print('No.'+str(cnt_code)+" Complete:%.4f%%" % (cnt_code/stock_download_len*100),end="\r")
        continue
    
    price_inf = jq.get_price(code, 
                 start_date='2011-01-01', 
                 end_date='2018-12-31', 
                 frequency='daily', 
                 fields=['close'], 
                 skip_paused=False, 
                 fq='pre', 
                 count=None)
    
    # 价格中出现零，剔除
    price_zero_num = len(price_inf[price_inf['close']==0])
    if price_zero_num != 0:
        continue
    
    price_inf['close_1'] = price_inf.shift(1)
    price_inf['return'] = np.log(price_inf['close']/price_inf['close_1'])
    price_inf = price_inf.dropna(how="any",axis=0)
    price_save_df = price_inf[['return']]
    
    # 记录收益率为0的天数和，停牌过多，剔除
    return_zero_num = len(price_save_df[price_save_df['return']==0])
    if return_zero_num > trade_days_len/3.8: # 2年的时间都处于停牌状态，剔除
        continue

    # 保存
    cnt_code += 1
    used_stock_list.append(code)
    price_save_df.to_csv('E:/Stock_Data/stock_return_data/'+code+'.csv',index=True)
    print('No.'+str(cnt_code)+" Complete:%.4f%%" % (cnt_code/stock_download_len*100),end="\r")
    
pd.DataFrame(used_stock_list).to_csv('E:/Stock_Data/return_inf_used_stock.csv',index=False)

In [6]:
price_save_df.to_csv('E:/Stock_Data/stock_return_data/'+code+'.csv',index=True)
pd.DataFrame(used_stock_list).to_csv('E:/Stock_Data/return_inf_used_stock.csv',index=False)

In [17]:
cnt_code = 0
code_list_temp = os.listdir('E:/Stock_Data/stock_return_data')
code_list = [i[:-4] for i in code_list_temp]

already_code_list_temp = os.listdir('E:/Stock_Data/stock_return_data_monthly')
already_code_list = [i[:-4] for i in already_code_list_temp]

stock_download_len = len(code_list)
used_stock_list_monthly = []

for code in code_list: # 从code_list下载，已经下载过日度数据，收益率数据要求更多，不一定要完整年报
    
    if code in already_code_list:
        cnt_code += 1
        print('No.'+str(cnt_code)+" Complete:%.4f%%" % (cnt_code/stock_download_len*100),end="\r")
        continue
    
    price_inf = jq.get_bars(code, 97, unit='1M',
         fields=['close','date'],
         include_now=False, end_dt='2019-01-01', fq_ref_date=None)
    
    # 价格中出现零，剔除
    price_zero_num = len(price_inf[price_inf['close']==0])
    if price_zero_num != 0:
        continue
    
    price_inf_new = price_inf[['close']]
    price_inf_new.index = price_inf['date']
    
    price_inf_new['close_1'] = price_inf_new.shift(1)
    price_inf_new['return'] = np.log(price_inf_new['close']/price_inf_new['close_1'])
    price_inf_new = price_inf_new.dropna(how="any",axis=0)
    price_save_df = price_inf_new[['return']]
    
    # 记录收益率为0的天数和，停牌过多，剔除
#     return_zero_num = len(price_save_df[price_save_df['return']==0])
#     if return_zero_num > trade_days_len/3.8: # 2年的时间都处于停牌状态，剔除
#         continue

    # 保存
    cnt_code += 1
    used_stock_list_monthly.append(code)
    price_save_df.to_csv('E:/Stock_Data/stock_return_data_monthly/'+code+'.csv',index=True)
    print('No.'+str(cnt_code)+" Complete:%.4f%%" % (cnt_code/stock_download_len*100),end="\r")
    
pd.DataFrame(used_stock_list_monthly).to_csv('E:/Stock_Data/return_inf_used_stock_monthly.csv',index=False)

In [5]:
for index_name in ['SZZS','SZCZ','CYBZ']:
    if index_name == 'SZZS':
        code = '000001.XSHG'
    if index_name == 'SZCZ':
        code = '399001.XSHE'
    if index_name == 'CYBZ':
        code = '399006.XSHE'

    price_inf = jq.get_bars(code, 97, unit='1M',
             fields=['close','date'],
             include_now=False, end_dt='2019-01-01', fq_ref_date=None)

    # 价格中出现零，剔除
    price_zero_num = len(price_inf[price_inf['close']==0])
    if price_zero_num != 0:
        continue

    price_inf_new = price_inf[['close']]
    price_inf_new.index = price_inf['date']

    price_inf_new['close_1'] = price_inf_new.shift(1)
    price_inf_new['return'] = np.log(price_inf_new['close']/price_inf_new['close_1'])
    price_inf_new = price_inf_new.dropna(how="any",axis=0)
    price_save_df = price_inf_new[['return']]

    # 记录收益率为0的天数和，停牌过多，剔除
    #     return_zero_num = len(price_save_df[price_save_df['return']==0])
    #     if return_zero_num > trade_days_len/3.8: # 2年的时间都处于停牌状态，剔除
    #         continue

    # 保存
    price_save_df.to_csv('E:/Stock_Data/stock_return_mon_index/'+index_name+'.csv',index=True)